# Temperature dependence of Hodgkin & Huxley action potentials

![image](https://www.gardeningknowhow.com/wp-content/uploads/2021/05/chili-pepper-400x300.jpg)

## Step 1: Setup

In [ ]:
# Setup inline plotting
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# For Google Colab, this line installs NEURON
!pip install neuron quantities

In [ ]:
# Fetch mechanisms
# Uncomment this line if on google colab
#!git clone https://github.com/ABL-Lab/NSC6084-A25.git

In [ ]:
# Compile the mechanisms
# Note: recompiled mechanisms will not take effect until neuron is imported or the jupyter kernel is restarted

# Uncomment this line if on google colab
#!nrnivmodl ./NSC6084-A25/Sept16/mechanisms
# Uncomment this line if running locally
!nrnivmodl mechanisms

In [ ]:
# We will let this library handle unit conversion for us
import quantities as pq
from quantities import um, nS, mV, cm, ms, nA, S, uF, Hz, degrees, s, MOhm

In [ ]:
# Import and initialize NEURON
import neuron
from neuron import h
h.load_file("stdrun.hoc")

In [ ]:
# Import other modules we need
import numpy as np

## Step 2: Define the circuit
We will use a single compartment, called a "Section" (more on that in next lectures). <br>
It has a cylindrical geometry with length "L" and a diameter "diam", and a specific capacitance "cm" (capacitance per area) <br>
**Unit conversion is a common source of error, so we will be explicit with our units.** 

In [ ]:
soma = h.Section()

### Query NEURON for the expected units for soma.L & soma.diam

In [ ]:
[h.units(x) for x in ["L", "diam"]]

In [ ]:
soma.L = 10 * um
soma.diam =  10 * um

In [ ]:
volume = soma(0.5).volume() * um**3

In [ ]:
area = soma(0.5).area() * um**2

In [ ]:
area

In [ ]:
volume

### Assign the membrane capacitance "everywhere"

In [ ]:
h.units("cm")  # Query the expected units

In [ ]:
specific_membrane_capacitance = 1 * uF/cm**2

In [ ]:
for sec in soma.wholetree():
    sec.cm = specific_membrane_capacitance #  specific membrane capacitance (micro Farads / cm^2)
    sec.Ra = 100

### Add the Hodgkin-Huxley conductances

In [ ]:
# This model includes the transient Na+, persistent K+ and the leak conductances
soma.insert("hh")

That's almost too easy!

### Parametize the leak conductance G = 1/R

In [ ]:
G = 0.1 * nS  # R = 1/G in our RC circuit

In [ ]:
v_rest = -70*mV

In [ ]:
tau_m = soma(0.5).cm / soma(0.5).hh.gl

In [ ]:
tau_m = (specific_membrane_capacitance * area / G).rescale(ms)

In [ ]:
tau_m

In [ ]:
# Assign the leak conductance everywhere
for seg in soma:
    seg.hh.gl = (G/area).rescale(S/cm**2)  # Compute specific conductance, and rescale to units of 'S/cm2'
    seg.hh.el = -54.3

### Inspect our parameters

In [ ]:
soma.psection()

In [ ]:
soma.nseg

### Add a current injection

In [ ]:
stim = h.IClamp(soma(0.5))

In [ ]:
stim.delay = 200 * ms  # Inject current 500ms after the start of the simulation 
stim.dur = 600 * ms  # stop injecting current at 520ms 
stim.amp = 0.013 * nA  # Inject 0.1 nA of current

## Step 3: Run the simulation

### Define recordings of simulation variables

In [ ]:
soma_v = h.Vector().record(soma(0.5)._ref_v)
t = h.Vector().record(h._ref_t)

In [ ]:
# Record hh gating variables
hh_vars = ['h', 'm', 'n', 'gna', 'gk']
hh_recordings = {}
for var in hh_vars:
    ref = getattr(soma(0.5).hh, "_ref_"+var )
    hh_recordings[var] = h.Vector().record(ref) 

In [ ]:
hh_recordings

In [ ]:
temp_range = [3.6, 6.3, 9.8, 13.3, 20.2]
stim.amp = 0.02 * nA  # Inject 0.1 nA of current
stim.dur = 20 * ms

In [ ]:
def run_sim(temp=6.3):
    h.celsius = temp
    h.finitialize( float(v_rest) )
    h.continuerun( float(1000 * ms) )
    return np.array(t), np.array(soma_v)

In [ ]:
plt.figure()
for temp in temp_range:
    a_t, v = run_sim(temp)
    plt.plot(a_t, v, label='%.1f$^\circ$C' % temp)
plt.axis([202,207,-80,50])
#plt.axis([0.01, 0.02, 0, 100])
plt.legend()
plt.xlabel("t (ms)")
plt.ylabel("v (mV)")

## Now it's your turn!

### **Activity 1** - Investigate the HH Na+ ion currents with voltage clamp



In [ ]:
# zero the current clamp to disable it
stim.amp = 0

### Add voltage clamp & define holding voltages

In [ ]:
vclamp = h.SEClamp(soma(0.5))

In [ ]:
vclamp.dur1 = 200 * ms  # vclamp of amp1 until dur1 after the start of the simulation 
vclamp.dur2 = 700 * ms  # start amp2 at dur1 and apply until dur1+dur2
vclamp.amp1 = v_rest #
vclamp.amp2 = -40*mV # holding voltage
vclamp.amp3 = v_rest # 

In [ ]:
(1/G).rescale(MOhm) # What is Rin in MOhm?

In [ ]:
vclamp.rs = 0.01 * MOhm # The clamp series resistance should be < 1/100 Rin

### Record the voltage clamp current

In [ ]:
vclamp_i = h.Vector().record(vclamp._ref_i)

### Define a function to run simulations

In [ ]:
def run_sim(holding_voltage, temp=6.3):
    h.celsis = temp
    
    # TODO
    
    pass

### Plot the current as a function of voltage step

### **Activity 2** - Investigate the temperature dependence of HH Na+ ion currents with voltage clamp




Plot the Na+ current using voltage clamp for a range of temperatures.  What is happening to the current as the temperature is lowered?

